## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import sqlite3
import zipfile

In [3]:
!cd ...

In [4]:
!ls

awesome.gif
CONTRIBUTING.md
LICENSE.md
movie_data_erd.jpeg
README.md
student.ipynb
student-Copy1.ipynb
zippedData
zippedDataim.db


In [5]:
# Extract IMDb SQL .db file
with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:
    # Extract all contents of .zip file into current directory
    zipObj.extractall(path='./zippedData/')
    
# Create connection to IMDb DB
con = sqlite3.connect('./zippedData/im.db')


In [6]:
# con = sqlite3.connect("zippedData/im.db/im.db")
# cursor = con.cursor()

In [7]:
schema_df = pd.read_sql("""
        SELECT *
        FROM sqlite_master """, con)

In [8]:
# Understanding the structure of the database
schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [9]:
# Looking at movie_basics
pd.read_sql(""" 
            SELECT * 
            FROM movie_basics
            LIMIT 2
            """, con)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


In [10]:
# Looking at movie_ratings
pd.read_sql(""" 
            SELECT *
            FROM movie_ratings
            LIMIT 2
            """, con)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


In [11]:
# inner joining movie_ratings and movie_basics
df_combined = pd.read_sql(""" 
            SELECT *
            FROM movie_ratings
            JOIN movie_basics
            ON movie_basics.movie_id = movie_ratings.movie_id
            """, con)

In [12]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   averagerating    73856 non-null  float64
 2   numvotes         73856 non-null  int64  
 3   movie_id         73856 non-null  object 
 4   primary_title    73856 non-null  object 
 5   original_title   73856 non-null  object 
 6   start_year       73856 non-null  int64  
 7   runtime_minutes  66236 non-null  float64
 8   genres           73052 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.1+ MB


In [13]:
# Dropping missing values in genres column
df_combined.dropna(subset = ['genres'], inplace = True)

In [14]:
df_combined.head()

,movie_id,averagerating,numvotes,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,tt10384606,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,tt1042974,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,tt1060240,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"


In [15]:
# pulling movie_budgets csv
df_budget = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [16]:


# replacing $ and , characters in production_budget column
df_budget.production_budget = df_budget.production_budget.str.replace('$','').str.replace(',','')

# casting to integer so we can filter based on production budget is 
df_budget.production_budget = df_budget.production_budget.astype(int)
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,"$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,"$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,350000000,"$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,"$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,"$620,181,382","$1,316,721,747"


In [17]:
# slicing out high budget films
df_high_budget = df_budget[df_budget.production_budget > 50000000]


In [18]:
High budget subset 

SyntaxError: invalid syntax (<ipython-input-18-15d540595b2d>, line 1)

In [40]:
# There are 16 different movies with the same title
df_high_budget[df_high_budget.duplicated(subset = ['movie'], keep = False) == True].sort_values('movie')


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
297,98,"Aug 7, 2015",Fantastic Four,120000000,"$56,117,548","$167,849,187"
484,85,"Jul 8, 2005",Fantastic Four,87500000,"$154,696,080","$333,132,750"
140,41,"May 16, 2014",Godzilla,160000000,"$200,676,069","$529,076,069"
273,74,"May 19, 1998",Godzilla,125000000,"$136,314,294","$376,000,000"
383,84,"Jul 25, 2014",Hercules,100000000,"$72,688,614","$243,388,614"
707,8,"Jun 13, 1997",Hercules,70000000,"$99,112,101","$250,700,000"
650,51,"Apr 16, 1999",Life,75000000,"$64,062,587","$73,521,587"
965,66,"Mar 24, 2017",Life,58000000,"$30,234,022","$100,929,666"
38,39,"May 14, 2010",Robin Hood,210000000,"$105,487,148","$322,459,006"
408,9,"Nov 21, 2018",Robin Hood,99000000,"$30,824,628","$84,747,441"


In [42]:
# Discovering that there are about 4 thousand of non unique titles in df_combined
len(df_combined.primary_title) - len(df_combined.primary_title.unique())

3804

In [54]:
# df_combined[df_combined['primary_title'] == "Robin Hood"]
df_combined.head()

,movie_id,averagerating,numvotes,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,tt10384606,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,tt1042974,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,tt1060240,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"


In [55]:
#Creating column "title_year" in table df_combined: to use it to merge tables without unnecessary data loss
df_combined['title_year'] = df_combined.apply(lambda row: row.primary_title + " " + str(row.start_year), axis=1)




In [72]:
df_combined.head(2)

,movie_id,averagerating,numvotes,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,title_year
0,tt10356526,8.3,31,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance,Laiye Je Yaarian 2019
1,tt10384606,8.9,559,tt10384606,Borderless,Borderless,2019,87.0,Documentary,Borderless 2019


In [70]:
#Creating column "title_year" in table df_high_budget: to use it to merge tables without unnecessary data loss
df_high_budget['title_year'] = df_high_budget.apply(lambda row: row.movie + " " + row.release_date[-5:], axis=1)

<ipython-input-70-dea0f179dc77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_budget['title_year'] = df_high_budget.apply(lambda row: row.movie + " " + row.release_date[-5:], axis=1)


In [71]:
df_high_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_year
0,1,"Dec 18, 2009",Avatar,425000000,"$760,507,625","$2,776,345,279",Avatar 2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,"$241,063,875","$1,045,663,875",Pirates of the Caribbean: On Stranger Tides 2011
2,3,"Jun 7, 2019",Dark Phoenix,350000000,"$42,762,350","$149,762,350",Dark Phoenix 2019
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,"$459,005,868","$1,403,013,963",Avengers: Age of Ultron 2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,"$620,181,382","$1,316,721,747",Star Wars Ep. VIII: The Last Jedi 2017


In [89]:
#cleaning columns not needed from analysis
df_combined.drop(columns=['movie_id', 'original_title', 'primary_title', 'runtime_minutes', 'start_year'], axis=1, inplace = True)

In [90]:
#cleaning columns not needed from analysis
df_high_budget.drop(columns=['id', 'release_date'], inplace = True)

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [95]:
df_high_budget.set_index(['title_year'])

,movie,production_budget,domestic_gross,worldwide_gross
title_year,,,,
Avatar 2009,Avatar,425000000,"$760,507,625","$2,776,345,279"
Pirates of the Caribbean: On Stranger Tides 2011,Pirates of the Caribbean: On Stranger Tides,410600000,"$241,063,875","$1,045,663,875"
Dark Phoenix 2019,Dark Phoenix,350000000,"$42,762,350","$149,762,350"
Avengers: Age of Ultron 2015,Avengers: Age of Ultron,330600000,"$459,005,868","$1,403,013,963"
Star Wars Ep. VIII: The Last Jedi 2017,Star Wars Ep. VIII: The Last Jedi,317000000,"$620,181,382","$1,316,721,747"
...,...,...,...,...
"Me, Myself & Irene 2000","Me, Myself & Irene",51000000,"$90,570,999","$149,270,999"
Barnyard: The Original Party Animals 2006,Barnyard: The Original Party Animals,51000000,"$72,779,000","$118,583,844"
Alpha 2018,Alpha,51000000,"$35,851,379","$99,624,873"


In [108]:
df_combined.reset_index()
df_high_budget.reset_index()

,index,movie,production_budget,domestic_gross,worldwide_gross,title_year
0,0,Avatar,425000000,"$760,507,625","$2,776,345,279",Avatar 2009
1,1,Pirates of the Caribbean: On Stranger Tides,410600000,"$241,063,875","$1,045,663,875",Pirates of the Caribbean: On Stranger Tides 2011
2,2,Dark Phoenix,350000000,"$42,762,350","$149,762,350",Dark Phoenix 2019
3,3,Avengers: Age of Ultron,330600000,"$459,005,868","$1,403,013,963",Avengers: Age of Ultron 2015
4,4,Star Wars Ep. VIII: The Last Jedi,317000000,"$620,181,382","$1,316,721,747",Star Wars Ep. VIII: The Last Jedi 2017
...,...,...,...,...,...,...
1076,1076,"Me, Myself & Irene",51000000,"$90,570,999","$149,270,999","Me, Myself & Irene 2000"
1077,1077,Barnyard: The Original Party Animals,51000000,"$72,779,000","$118,583,844",Barnyard: The Original Party Animals 2006
1078,1078,Alpha,51000000,"$35,851,379","$99,624,873",Alpha 2018
1079,1079,Deck the Halls,51000000,"$35,093,569","$46,815,807",Deck the Halls 2006


In [109]:
df_combined.title_year = df_combined.title_year.astype(str)
df_high_budget.title_year = df_high_budget.title_year.astype(str)

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [110]:

df_newdata = pd.merge(df_combined, df_high_budget, left_on='title_year', right_on='title_year')

In [111]:
#DOES NOT MERGE CORRECTLY
df_newdata.head()

,averagerating,numvotes,genres,title_year,movie,production_budget,domestic_gross,worldwide_gross
